このノートブックでは、OpenAIと[MongoDB Atlas vector search](https://www.mongodb.com/products/platform/atlas-vector-search)を使用してセマンティック検索アプリケーションを構築する方法を説明します。

In [ ]:
!pip install pymongo openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


# ステップ1: 環境のセットアップ

これには2つの前提条件があります：

1.   **MongoDB Atlasクラスター**: 永続的に無料のMongoDB Atlasクラスターを作成するには、まずMongoDB Atlasアカウントをお持ちでない場合は作成する必要があります。[MongoDB Atlasウェブサイト](https://www.mongodb.com/atlas/database)にアクセスし、「Register」をクリックしてください。[MongoDB Atlas](https://account.mongodb.com/account/login)ダッシュボードにアクセスし、クラスターをセットアップしてください。集約パイプラインで`$vectorSearch`オペレーターを活用するには、MongoDB Atlas 6.0.11以上を実行する必要があります。このチュートリアルは無料クラスターを使用して構築できます。デプロイメントをセットアップする際、データベースユーザーとネットワーク接続のルールを設定するよう求められます。ユーザー名とパスワードを安全な場所に保存し、クラスターが適切に接続できるよう正しいIPアドレスルールを設定してください。開始にあたってさらにヘルプが必要な場合は、[MongoDB Atlasのチュートリアル](https://www.mongodb.com/basics/mongodb-atlas-tutorial)をご確認ください。

2. **OpenAI APIキー** OpenAIキーを作成するには、アカウントを作成する必要があります。アカウントを作成したら、[OpenAIプラットフォーム](https://platform.openai.com/)にアクセスしてください。画面右上のプロフィールアイコンをクリックしてドロップダウンメニューを表示し、「View API keys」を選択してください。

In [ ]:
import getpass

MONGODB_ATLAS_CLUSTER_URI = getpass.getpass("MongoDB Atlas Cluster URI:")
OPENAI_API_KEY = getpass.getpass("OpenAI API Key:")


MongoDB Atlas Cluster URI:··········
OpenAI API Key:··········


注意：上記の手順を実行した後、認証情報の入力を求められます。

このチュートリアルでは、[MongoDB sample dataset](https://www.mongodb.com/docs/atlas/sample-data/)を使用します。Atlas UIを使用してサンプルデータセットを読み込んでください。"sample_mflix"データベースを使用します。このデータベースには"movies"コレクションが含まれており、各ドキュメントにはtitle、plot、genres、cast、directorsなどのフィールドが含まれています。

In [ ]:
import openai
import pymongo

client = pymongo.MongoClient(MONGODB_ATLAS_CLUSTER_URI)
db = client.sample_mflix
collection = db.movies

openai.api_key = OPENAI_API_KEY

In [ ]:
ATLAS_VECTOR_SEARCH_INDEX_NAME = "default"
EMBEDDING_FIELD_NAME = "embedding_openai_nov19_23"

# ステップ 2: 埋め込み生成関数のセットアップ

In [ ]:
model = "text-embedding-3-small"
def generate_embedding(text: str) -> list[float]:
    return openai.embeddings.create(input = [text], model=model).data[0].embedding


# ステップ3: 埋め込みの作成と保存

サンプルデータセット sample_mflix.movies の各ドキュメントは映画に対応しています。「plot」フィールドのデータに対してベクトル埋め込みを作成し、データベースに保存する操作を実行します。意図に基づく類似性検索を実行するには、OpenAI埋め込みエンドポイントを使用してベクトル埋め込みを作成することが必要です。

In [ ]:
from pymongo import ReplaceOne

# Update the collection with the embeddings
requests = []

for doc in collection.find({'plot':{"$exists": True}}).limit(500):
  doc[EMBEDDING_FIELD_NAME] = generate_embedding(doc['plot'])
  requests.append(ReplaceOne({'_id': doc['_id']}, doc))

collection.bulk_write(requests)

BulkWriteResult({'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 50, 'nModified': 50, 'nRemoved': 0, 'upserted': []}, acknowledged=True)

上記を実行した後、"movies"コレクション内のドキュメントには、title、plot、genres、cast、directorsなどの既存のフィールドに加えて、`EMBEDDDING_FIELD_NAME`変数で定義された"embedding"という追加フィールドが含まれるようになります。

注意：時間の都合上、これを500ドキュメントのみに制限しています。sample_mflixデータベースの23,000以上のドキュメントからなる全データセットに対してこれを実行したい場合は、少し時間がかかります。代替案として、[sample_mflix.embedded_movies collection](https://www.mongodb.com/docs/atlas/sample-data/sample-mflix/#sample_mflix.embedded_movies)を使用することができます。このコレクションには、OpenAIの`text-embedding-3-small`埋め込みモデルを使用して作成された埋め込みを含む事前入力済みの`plot_embedding`フィールドが含まれており、Atlas Searchのベクトル検索機能で使用できます。

# ステップ4: ベクトル検索インデックスの作成

このコレクションにAtlas Vector Search Indexを作成します。これにより、セマンティック検索を支える近似KNN検索を実行できるようになります。
このインデックスを作成する2つの方法について説明します - Atlas UIとMongoDB pythonドライバーを使用する方法です。

(オプション) [ドキュメント: ベクトル検索インデックスの作成](https://www.mongodb.com/docs/atlas/atlas-search/field-types/knn-vector/)

[Atlas UI](cloud.mongodb.com)にアクセスして、[こちら](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-tutorial/#create-the-atlas-vector-search-index)で説明されている手順を使用してAtlas Vector Searchインデックスを作成してください。値が1536の'dimensions'フィールドは、OpenAIのtext-embedding-ada002に対応しています。

Atlas UIのJSONエディターで以下の定義を使用してください。

```
{
  "mappings": {
    "dynamic": true,
    "fields": {
      "embedding": {
        "dimensions": 1536,
        "similarity": "dotProduct",
        "type": "knnVector"
      }
    }
  }
}
```

（オプション）代替として、[pymongoドライバーを使用してこれらのベクトル検索インデックスをプログラム的に作成する](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.create_search_index)ことができます。
以下のセルで示されているpythonコマンドはインデックスを作成します（これはMongoDB用Pythonドライバーの最新バージョンとMongoDBサーバーバージョン7.0以降のAtlasクラスターでのみ動作します）。

In [ ]:
collection.create_search_index(
    {"definition":
        {"mappings": {"dynamic": True, "fields": {
            EMBEDDING_FIELD_NAME : {
                "dimensions": 1536,
                "similarity": "dotProduct",
                "type": "knnVector"
                }}}},
     "name": ATLAS_VECTOR_SEARCH_INDEX_NAME
    }
)

'default'

# ステップ5: データをクエリする

ここでのクエリの結果は、キーワード検索に基づくのではなく、クエリ文字列で指定されたテキストと意味的に類似したプロットを持つ映画を見つけます。

（オプション）[ドキュメント: ベクトル検索クエリの実行](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/)

In [ ]:

def query_results(query, k):
  results = collection.aggregate([
    {
        '$vectorSearch': {
            "index": ATLAS_VECTOR_SEARCH_INDEX_NAME,
            "path": EMBEDDING_FIELD_NAME,
            "queryVector": generate_embedding(query),
            "numCandidates": 50,
            "limit": 5,
        }
    }
    ])
  return results

In [ ]:
query="imaginary characters from outerspace at war with earthlings"
movies = query_results(query, 5)

for movie in movies:
    print(f'Movie Name: {movie["title"]},\nMovie Plot: {movie["plot"]}\n')